https://dl.acm.org/doi/10.1145/3656456 Numerical Fuzz: A Type System for Rounding Error Analysis


https://malyzajko.github.io/papers/sas2023a.pdf Modular analysis eva

In [ ]:
from kdrag.all import *

x,y,z = smt.Reals("x y z")
#rnd = kd.define("rnd", [x,y], kd.QExists([z], 0 <= z, z <= 0.1, y == x*(1 + z)))

# could do smt.Function(smt.RealSort(), Fl). It's kind of a smart constructor
rnd = smt.Function("rnd", smt.RealSort(), smt.RealSort())
err = smt.Function("err", smt.RealSort(), smt.RealSort())


# we could condition the equation on x being in allowed range to model undefinedness
#rnd_ax = kd.axiom(kd.QForAll([x], -1 <= x, x <= 1, smt.And(0 <= err(x), err(x) <= 0.1, rnd(x) == x + err(x))))
rnd_ax = kd.axiom(smt.ForAll([x], smt.And(0 <= err(x), err(x) <= 0.1, rnd(x) == x*(1 + err(x)))))


Fl = kd.NewType("Fl", smt.RealSort(), strict=False)
fx,fy,fz = smt.Consts("fx fy fz", Fl)    
fl_add = kd.define("add", [fx,fy], Fl(rnd(fx.val + fy.val)))
kd.notation.add.register(Fl, fl_add)

fl_add_comm = kd.lemma(kd.QForAll([fx,fy], fx + fy == fy + fx), by=fl_add.defn)






In [ ]:
from kdrag.all import *

x,y,z = smt.Reals("x y z")

Fix = kd.NewType("Fix", smt.RealSort(), strict=False) # pred=lambda fx: smt.And(fx.val >= -1, fx.val <= 1)) 


err = smt.Function("err", smt.RealSort(), smt.RealSort())
err_bnd = kd.axiom(kd.QForAll([x], -1 <= x, x <= 1, smt.And(-1/smt.RealVal(2**8) <= err(x), err(x) < 1/smt.RealVal(2**8))))
rnd = kd.define("rnd", [x], Fix(x + err(x)))
#rnd = smt.Function("rnd", smt.RealSort(), Fix)
#rnd_ax = kd.axiom(kd.QForAll([x], rnd(x).val == x + err(x)))


fx,fy,fz = smt.Consts("fx fy fz", Fix)    
fix_add = kd.define("add", [fx,fy], rnd(fx.val + fy.val))
kd.notation.add.register(Fix, fix_add)

fl_add_comm = kd.lemma(kd.QForAll([fx,fy], fx + fy == fy + fx), by=fix_add.defn)
#smt.simplify(rnd_ax(smt.RealVal(0.1)).thm)


# It would be kind of nice to be able to explore this way,
#c = kd.Calc([], (fx + fx + fx).val)
#c.unfold(fix_add.defn, rnd.defn)

l = kd.Lemma(z == (fx + fx + fx).val)
l.unfold(fix_add, fix_add, rnd, rnd)
l.simp()

err3 = kd.define("err3", [fx], (fx + fx + fx).val - (3*fx.val)) 

l = kd.Lemma(kd.QForAll([fx], 0 <= fx.val, fx.val <= 0.1, err3(fx) <= 3/smt.RealVal(2**8)))
_fx = l.fixes()
l.intros()
l.unfold(err3)
l.unfold(fix_add, fix_add)
l.unfold(rnd, rnd)
l.simp()
l.auto(by=[err_bnd])
l.qed()

l = kd.lemma(kd.QForAll([fx], 0 <= fx.val, fx.val <= 0.1, err3(fx) <= 3/smt.RealVal(2**8)), by=[err3.defn, err_bnd, fix_add.defn, rnd.defn])


fix_mul = kd.define("mul", [fx,fy], rnd(fx.val * fy.val))
kd.notation.mul.register(Fix, fix_mul)

fx * fy



mul(fx, fy)